# List is a over-allocate array


```C
#ifndef Py_LIMITED_API
typedef struct {
    PyObject_VAR_HEAD
    /* Vector of pointers to list elements.  list[0] is ob_item[0], etc. */
    PyObject **ob_item;

    /* ob_item contains space for 'allocated' elements.  The number
     * currently in use is ob_size.
     * Invariants:
     *     0 <= ob_size <= allocated
     *     len(list) == ob_size
     *     ob_item == NULL implies ob_size == allocated == 0
     * list.sort() temporarily sets allocated to -1 to detect mutations.
     *
     * Items must normally not be NULL, except during construction when
     * the list is not yet visible outside the function that builds it.
     */
    Py_ssize_t allocated;
} PyListObject;
#endif

```

listobject.h：https://github.com/python/cpython/blob/949fe976d5c62ae63ed505ecf729f815d0baccfc/Include/listobject.h#L23

list 本质上是一个 over-allocate 的 array。其中，ob_item 是一个指针列表，里面的每一个指针都指向列表的元素。而 allocated 则存储了这个列表已经被分配的空间大小。

需要注意的是，allocated 与列表实际空间大小的区别。列表实际空间大小，是指 len(list) 返回的结果，即上述代码注释中的 ob_size，表示这个列表总共存储了多少个元素。实际情况下，为了优化存储结构，避免每次增加元素都要重新分配内存，列表预分配的空间 allocated 往往会大于 ob_size。

所以，它们的关系为：allocated >= len(list) = ob_size。

如果当前列表分配的空间已满（即 allocated == len(list)），则会向系统请求更大的内存空间，并把原来的元素全部拷贝过去。列表每次分配空间的大小，遵循下面的模式：
```txt
0, 4, 8, 16, 25, 35, 46, 58, 72, 88, ...
2
```

啥叫over-allocate呢？就是当底层数组容量满了而需要扩充的时候，python依据规则会扩充多个位置出来。比如初始化列表array=[1, 2, 3, 4]，向其中添加元素23，此时array对应的底层数组，扩充后的容量不是5，而是8。这就是over-allocate的意义，即扩充容量的时候会多分配一些存储空间。这样做的优点当然是提高了执行效率，否则每次添加元素，都要对底层数组进行扩充，效率是很低下的。另外，当列表存储的元素在变少时，python也会及时收缩底层的数组，避免造成内存浪费。第一次会扩充为4个元素的容量，占用32个字节，每个元素占用8个，注意这里就是每个元素占用8个字节。

In [1]:
l = []
print(l)
l.__sizeof__()

[]


40

In [2]:
l.append(1)
print(l)
l.__sizeof__()

[1]


72

In [3]:
l.append(2)
print(l)
l.__sizeof__()

[1, 2]


72

In [4]:
l.append(3)
print(l)
l.__sizeof__()

[1, 2, 3]


72

In [5]:
l.append(4)
print(l)
l.__sizeof__()

[1, 2, 3, 4]


72

In [6]:
l.append(5)
print(l)
l.__sizeof__()

[1, 2, 3, 4, 5]


104

In [33]:
l.remove(5)
print(l)
l.__sizeof__()

[1, 2, 3, 4]


104

In [7]:
l.remove(4)
print(l)
l.__sizeof__()

[1, 2, 3, 5]


104

In [8]:
l.remove(3)
print(l)
l.__sizeof__()

[1, 2, 5]


104

In [9]:
l.remove(2)
print(l)
l.__sizeof__()

[1, 5]


104

In [10]:
l.remove(1)
print(l)
l.__sizeof__()

[5]


72

**tuple从具体结构上来看，和list相似，本质上也是一个array，但是空间大小固定。**

Python的tuple做了许多优化来提升在程序中的效率。例如，**在tuple的大小不超过20时，Python就会把它缓存到一个内部的free list中，如果之后再创建相同的tuple，Python会直接从缓存中载入，从而提升程序效率**。




## 为什么旧的hash表中的元素越来越稀疏

旧哈希表的示意图
```txt
--+-------------------------------+
  | 哈希值 (hash)  键 (key)  值 (value)
--+-------------------------------+
0 |    hash0      key0    value0
--+-------------------------------+
1 |    hash1      key1    value1
--+-------------------------------+
2 |    hash2      key2    value2
--+-------------------------------+
. |           ...
__+_______________________________+


```

你会发现，它是一个 over-allocate 的 array，根据元素键（key）的哈希值，来计算其应该被插入位置的索引。 因此，假设我有下面这样一个字典：
`{'name': 'mike', 'dob': '1999-01-01', 'gender': 'male'}`

那么这个字典便会存储为类似下面的形式：
```txt
entries = [
['--', '--', '--']
[-230273521, 'dob', '1999-01-01'],
['--', '--', '--'],
['--', '--', '--'],
[1231236123, 'name', 'mike'],
['--', '--', '--'],
[9371539127, 'gender', 'male']
]

```

这里的’---‘，表示这个位置没有元素，但是已经分配了内存。 我们知道，当哈希表剩余空间小于 1/3 时，为了保证相关操作的高效性并避免哈希冲突，就会重新分配更大的内存。所以，当哈希表中的元素越来越多时，分配了内存但里面没有元素的位置，也会变得越来越多。这样一来，哈希表便会越来越稀疏。

而新哈希表的结构，改变了这一点，也大大提高了空间的利用率。新哈希表的结构如下所示：
```txt
Indices
----------------------------------------------------
None | index | None | None | index | None | index ...
----------------------------------------------------


Entries
--------------------
hash0   key0  value0
---------------------
hash1   key1  value1
---------------------
hash2   key2  value2
---------------------
        ...
---------------------
```
你可以看到，它把存储结构分成了 Indices 和 Entries 这两个 array，而’None‘代表这个位置分配了内存但没有元素。 我们同样还用上面这个例子，它在新哈希表中的存储模式，就会变为下面这样：
```python
indices = [None, 1, None, None, 0, None, 2]
entries = [
[1231236123, 'name', 'mike'],
[-230273521, 'dob', '1999-01-01'],
[9371539127, 'gender', 'male']
]
```
其中，Indices 中元素的值，对应 entries 中相应的索引。比如indices中的1，就对应着entries[1]，即’'dob': '1999-01-01'‘。 对比之下，我们会清晰感受到，新哈希表中的空间利用率，相比于旧哈希表有大大的提升。